In [196]:
import datetime
import pandas as pd

# Plan Parameters
plan_name = 'english3000_memory'
list_prefix = 'L'
start_date = datetime.date(2023, 8, 30)
total_lists = 91
skip_dates = ["2023-09-29", "2023-10-09", "2023-10-10", "2024-01-01"]
review_intervals = [0, 1, 2, 4, 7, 15]

# Create an empty DataFrame to store the plan
result_df = pd.DataFrame(columns=["Name", "Date", "List"])

# Generate the plan
cnt = 0
while cnt < total_lists:
    while (
        start_date.weekday() in {5, 6}
        or str(start_date) in skip_dates
    ):
        start_date += datetime.timedelta(days=1)
    
    cnt += 1
    list_name = f"{list_prefix}{cnt:02}"
    result_df = pd.concat(
        [result_df, pd.DataFrame([{
            "Name": list_name,
            "Date": start_date.strftime("%B %#d, %Y"),
            "List": []}])]
        , ignore_index=True)
    start_date += datetime.timedelta(days=1)

# Prepare dictionaries for easier access
list_dict = result_df.set_index('Name')['Date'].to_dict()
date_dict = {date: [] for date in result_df['Date']}
review_dict = {}

# Generate the review schedule
for idx, row in result_df.iterrows():
    list_id = row["Name"]
    list_date = list_dict[list_id]
    review_dict[list_id] = [list_date]
    for interval in review_intervals[1:]:
        next_idx = idx + interval
        if next_idx < len(result_df):
            review_dict[list_id].append(result_df.loc[next_idx, 'Date'])

# Populate date_dict with review information
for list_id, dates in review_dict.items():
    for date_i in dates:
        date_dict[date_i].append(int(list_id[1:]))

# Update the 'List' column in the DataFrame
for date, review_lists in date_dict.items():
    idx = result_df.loc[result_df['Date'] == date].index[0]
    result_df.loc[idx, 'List'] = ', '.join(map(str, review_lists))

# Save the generated plan to an Excel file
result_df.to_excel(f"./result/{plan_name}.xlsx", index=False)